### Electic Vehicle Data for Washington State

In [64]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math
from IPython.display import display, Math

pd.options.display.max_columns = None

In [4]:

ev_wa = pd.read_csv('Resources/Electric_Vehicle_Population_Data.csv')
print(ev_wa.shape, '\n')
ev_wa.info()
ev_wa.tail(2)

(257635, 17) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257635 entries, 0 to 257634
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         257635 non-null  object 
 1   County                                             257627 non-null  object 
 2   City                                               257627 non-null  object 
 3   State                                              257635 non-null  object 
 4   Postal Code                                        257627 non-null  float64
 5   Model Year                                         257635 non-null  int64  
 6   Make                                               257635 non-null  object 
 7   Model                                              257635 non-null  object 
 8   Electric Vehicle Type                              257635 n

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
257633,JTDKARFP9K,Whatcom,Bellingham,WA,98229.0,2019,TOYOTA,PRIUS PRIME (PHEV),Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25.0,0.0,40.0,177812232,POINT (-122.45486 48.7449),PUGET SOUND ENERGY INC||PUD NO 1 OF WHATCOM CO...,5.307300e+10
257634,7YAKRDDC1S,King,Seattle,WA,98121.0,2025,HYUNDAI,IONIQ 5,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,43.0,280545338,POINT (-122.34468 47.61578),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10


## Checking for Blank Cells and Cells with 0s

In [5]:
print(f'\nCells with value 0: {ev_wa.isnull().values.sum()}')  # Counts the number of Null cells
print(f'\nColumns with blank cells: : {ev_wa.columns[ev_wa.isna().any()].tolist()}\n\n')
ev_wa['Postal Code'] = ev_wa['Postal Code'].fillna(0)
ev_wa['Postal Code'] = ev_wa['Postal Code'].astype('int64')
ev_wa.info()



Cells with value 0: 659

Columns with blank cells: : ['County', 'City', 'Postal Code', 'Electric Range', 'Base MSRP', 'Legislative District', 'Vehicle Location', 'Electric Utility', '2020 Census Tract']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257635 entries, 0 to 257634
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         257635 non-null  object 
 1   County                                             257627 non-null  object 
 2   City                                               257627 non-null  object 
 3   State                                              257635 non-null  object 
 4   Postal Code                                        257635 non-null  int64  
 5   Model Year                                         257635 non-null  int64  
 6   Make                          

## Filtering data for only Washington state

In [6]:
print(f'EV DataFrame, includes all States in US: (Rows, Columns) {ev_wa.shape}\n')

# Filtering for Wahington State
ev_wa = ev_wa[ev_wa['State'] == 'WA']

print(f'EV DataFrame, includes only Washington State: (Rows, Columns) {ev_wa.shape}\n')

EV DataFrame, includes all States in US: (Rows, Columns) (257635, 17)

EV DataFrame, includes only Washington State: (Rows, Columns) (257038, 17)



## Modifying cell values and columns for analysis

In [7]:

ev_wa['Vehicle Type'] = ev_wa['Electric Vehicle Type'].str.extract(r'\((.*?)\)')

ev_wa = ev_wa.drop(['VIN (1-10)', 'Electric Vehicle Type', 'Legislative District', 'DOL Vehicle ID', 'Electric Utility', '2020 Census Tract'], axis = 1)

ev_wa.info()


<class 'pandas.core.frame.DataFrame'>
Index: 257038 entries, 0 to 257634
Data columns (total 12 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   County                                             257038 non-null  object 
 1   City                                               257038 non-null  object 
 2   State                                              257038 non-null  object 
 3   Postal Code                                        257038 non-null  int64  
 4   Model Year                                         257038 non-null  int64  
 5   Make                                               257038 non-null  object 
 6   Model                                              257038 non-null  object 
 7   Clean Alternative Fuel Vehicle (CAFV) Eligibility  257038 non-null  object 
 8   Electric Range                                     257035 non-null  float64
 9 

In [9]:
#Clean Alternative Fuel Vehicle (CAFV) Eligibility
ev_wa.rename({'Clean Alternative Fuel Vehicle (CAFV) Eligibility': 'CAFV Eligibility'}, axis = 1, inplace = True)

eligibility = {
    'Clean Alternative Fuel Vehicle Eligible':'Eligible',
    'Not eligible due to low battery range': 'Not eligible',
    'Eligibility unknown as battery range has not been researched': 'Unknown'
    }

ev_wa['CAFV Eligibility'] = ev_wa['CAFV Eligibility'].replace(eligibility)
ev_wa['CAFV Eligibility'].tail()

257630        Eligible
257631         Unknown
257632    Not eligible
257633    Not eligible
257634         Unknown
Name: CAFV Eligibility, dtype: object

#### Creating a combined column: 'Year_Make_Model'

In [10]:
ev_wa['Year_Make_Model'] = ev_wa['Model Year'].astype(str) + ' ' + ev_wa['Make'] + ' ' + ev_wa['Model']
#ev_wa.head()

#### Splitting 'Longitude', 'Latitude'

In [11]:
ev_zero_coords = ev_wa[ev_wa['Vehicle Location'].isna() | ev_wa['Vehicle Location'].isin(['', ' '])]

print(f' Number of Null values for Vehicle Location: {len(ev_zero_coords)}\n')
ev_zero_coords[['County', 'City', 'Vehicle Location']]

 Number of Null values for Vehicle Location: 8



,County,City,Vehicle Location
31777,Pacific,Long Beach,NaN
37238,Pacific,Long Beach,NaN
45389,Pacific,Long Beach,NaN
64507,Pacific,Long Beach,NaN
226210,Pacific,Long Beach,NaN
239956,Pacific,Long Beach,NaN
248631,Pacific,Long Beach,NaN
249335,Pacific,Long Beach,NaN


In [12]:
# List of indices with missing 'Vehicle Location'
target_indices = [31777, 37238, 45389, 64507, 226210, 239956, 248631, 249335]

# Get the reference 'Vehicle Location' value for Long Beach
ref_location = ev_wa.loc[
    ev_wa['City'] == 'Long Beach', 'Vehicle Location'
].dropna().unique()

# Check the find
print(f'Reference Vehicle Location for Long Beach:  {ref_location}\n')

# Fill missing or blank entries with that reference
if len(ref_location) > 0:
    ev_wa.loc[
        (ev_wa['City'] == 'Long Beach') &
        (ev_wa['Vehicle Location'].isna() | ev_wa['Vehicle Location'].isin(['', ' '])),
        'Vehicle Location'
    ] = ref_location[0]

# Display 'City' and 'Vehicle Location' for the rows (indicies)
ev_wa.loc[target_indices, ['City', 'Vehicle Location']]    


Reference Vehicle Location for Long Beach:  ['POINT (-124.05439 46.35232)']



,City,Vehicle Location
31777,Long Beach,POINT (-124.05439 46.35232)
37238,Long Beach,POINT (-124.05439 46.35232)
45389,Long Beach,POINT (-124.05439 46.35232)
64507,Long Beach,POINT (-124.05439 46.35232)
226210,Long Beach,POINT (-124.05439 46.35232)
239956,Long Beach,POINT (-124.05439 46.35232)
248631,Long Beach,POINT (-124.05439 46.35232)
249335,Long Beach,POINT (-124.05439 46.35232)


In [13]:
# Extract latitude and longitude from 'Vehicle Location' column using regex

ev_wa['Longitude'] = ev_wa['Vehicle Location'].str.extract(r'\((-?\d+\.\d+)\s(-?\d+\.\d+)\)')[0]
ev_wa['Latitude'] = ev_wa['Vehicle Location'].str.extract(r'\((-?\d+\.\d+)\s(-?\d+\.\d+)\)')[1]

# Convert 'Latitude' and 'Longitude' columns to float type
ev_wa['Latitude'] = ev_wa['Latitude'].astype(float)
ev_wa['Longitude'] = ev_wa['Longitude'].astype(float)

#Dropping column 'Vehicle Location'
ev_wa = ev_wa.drop(['Vehicle Location'], axis = 1)

ev_wa[50:55]

,County,City,State,Postal Code,Model Year,Make,Model,CAFV Eligibility,Electric Range,Base MSRP,Vehicle Type,Year_Make_Model,Longitude,Latitude
51,Thurston,Olympia,WA,98506,2014,FIAT,500,Eligible,87.0,0.0,BEV,2014 FIAT 500,-122.87741,47.05997
52,Snohomish,Mukilteo,WA,98275,2020,JAGUAR,I-PACE,Eligible,234.0,0.0,BEV,2020 JAGUAR I-PACE,-122.29196,47.89908
53,Snohomish,Edmonds,WA,98026,2017,NISSAN,LEAF,Eligible,107.0,0.0,BEV,2017 NISSAN LEAF,-122.31768,47.87166
54,Snohomish,Lynnwood,WA,98036,2020,KIA,NIRO,Eligible,239.0,0.0,BEV,2020 KIA NIRO,-122.29245,47.82557
55,Whitman,Pullman,WA,99163,2018,TESLA,MODEL 3,Eligible,215.0,0.0,BEV,2018 TESLA MODEL 3,-117.18147,46.73015


In [14]:
ev_wa.rename({'Model Year': 'Year'}, axis = 1, inplace = True)

# Rearrange order of columns
ev_wa = ev_wa[['State', 'County', 'City', 'Postal Code', 'Latitude', 'Longitude',
         'Year', 'Make', 'Model', 'Vehicle Type', 'Electric Range',
         'CAFV Eligibility', 'Year_Make_Model', 'Base MSRP']]
ev_wa.head(3)

,State,County,City,Postal Code,Latitude,Longitude,Year,Make,Model,Vehicle Type,Electric Range,CAFV Eligibility,Year_Make_Model,Base MSRP
0,WA,Yakima,Yakima,98901,46.60464,-120.50729,2019,TESLA,MODEL 3,BEV,220.0,Eligible,2019 TESLA MODEL 3,0.0
1,WA,Kitsap,Port Orchard,98367,47.50524,-122.68471,2024,JEEP,WRANGLER,PHEV,21.0,Not eligible,2024 JEEP WRANGLER,0.0
2,WA,Snohomish,Lynnwood,98036,47.82557,-122.29245,2022,KIA,NIRO,PHEV,26.0,Not eligible,2022 KIA NIRO,0.0


In [15]:
# Copy the Washington DataFrame; drop redundant columns
ev = ev_wa.copy()
ev = ev.drop(['State', 'City', 'Postal Code'], axis = 1)
ev.head(3)

,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,Electric Range,CAFV Eligibility,Year_Make_Model,Base MSRP
0,Yakima,46.60464,-120.50729,2019,TESLA,MODEL 3,BEV,220.0,Eligible,2019 TESLA MODEL 3,0.0
1,Kitsap,47.50524,-122.68471,2024,JEEP,WRANGLER,PHEV,21.0,Not eligible,2024 JEEP WRANGLER,0.0
2,Snohomish,47.82557,-122.29245,2022,KIA,NIRO,PHEV,26.0,Not eligible,2022 KIA NIRO,0.0


#### Population Data in Washington State

In [18]:
pd.options.display.max_columns = None

pop = pd.read_csv('Resources/WAOFM_-_April_1_-_Population_by_State__County_and_City__1990_to_Present.csv')
pop.head(3)

,SEQUENCE,FILTER,COUNTY,JURISDICTION,POP_1990,POP_1991,POP_1992,POP_1993,POP_1994,POP_1995,POP_1996,POP_1997,POP_1998,POP_1999,POP_2000,POP_2001,POP_2002,POP_2003,POP_2004,POP_2005,POP_2006,POP_2007,POP_2008,POP_2009,POP_2010,POP_2011,POP_2012,POP_2013,POP_2014,POP_2015,POP_2016,POP_2017,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023,POP_2024,POP_2025
0,1,1,Adams,Adams County,13603.0,13823.0,14063.0,14335.0,14679.0,15030.0,15323.0,15698.0,15879.0,16151.0,16428,16699,16911,17081,17489,17643,17690,17959,18214,18421,18728,18849,19013,19172,19309,19451,19643,20068,20200,20335,20613,20900,21100,21200,21475,21550
1,2,2,Adams,Unincorporated Adams County,6466.0,6698.0,6776.0,7009.0,7162.0,7303.0,7530.0,7598.0,7647.0,7815.0,7905,8037,8193,8324,8451,8507,8587,8682,8742,8799,8818,8866,8925,8993,9024,9093,9205,9250,9301,9375,9472,9575,9575,9585,9628,9640
2,3,3,Adams,Incorporated Adams County,7137.0,7125.0,7287.0,7326.0,7517.0,7727.0,7793.0,8100.0,8232.0,8336.0,8523,8662,8718,8757,9038,9136,9103,9277,9472,9622,9910,9983,10088,10179,10285,10358,10438,10818,10899,10960,11141,11325,11525,11615,11847,11910


In [19]:
# Retaining only the latest population data (2025)

pop = pop.drop(['SEQUENCE', 'FILTER', 'POP_1990', 'POP_1991', 'POP_1992', 'POP_1993', 'POP_1994', 'POP_1995', 'POP_1996',
                'POP_1997', 'POP_1998', 'POP_1999', 'POP_2000', 'POP_2001', 'POP_2002', 'POP_2003', 'POP_2004', 'POP_2005',
                'POP_2006', 'POP_2007', 'POP_2008', 'POP_2009', 'POP_2010', 'POP_2011', 'POP_2012', 'POP_2013', 'POP_2014',
                'POP_2015', 'POP_2016', 'POP_2017', 'POP_2018', 'POP_2019', 'POP_2020', 'POP_2021', 'POP_2022', 'POP_2023', 'POP_2024'], axis = 1)
print(pop.shape)
pop.head()

(409, 3)


,COUNTY,JURISDICTION,POP_2025
0,Adams,Adams County,21550
1,Adams,Unincorporated Adams County,9640
2,Adams,Incorporated Adams County,11910
3,Adams,Hatton,165
4,Adams,Lind,535


In [20]:

# Extracting the total population of each county
pop_county = (
    pop[pop['JURISDICTION'].str.fullmatch(r'.+\sCounty')]  
      .drop_duplicates(subset='COUNTY', keep='first')     
      [['COUNTY', 'POP_2025']]                           
)

pop_county = pop_county.reset_index(drop=True)
pop_county[:5]


,COUNTY,POP_2025
0,Adams,21550
1,Asotin,22750
2,Benton,219625
3,Chelan,82900
4,Clallam,78650


#### Merge EV data with County Population data

In [21]:
pop_ev = pd.merge(ev, pop_county, left_on=['County'], right_on=['COUNTY'], how='left')
pop_ev = pop_ev.drop(columns='COUNTY')
print(pop_ev.shape)
pop_ev.head(3)

(257038, 12)


,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,Electric Range,CAFV Eligibility,Year_Make_Model,Base MSRP,POP_2025
0,Yakima,46.60464,-120.50729,2019,TESLA,MODEL 3,BEV,220.0,Eligible,2019 TESLA MODEL 3,0.0,264650
1,Kitsap,47.50524,-122.68471,2024,JEEP,WRANGLER,PHEV,21.0,Not eligible,2024 JEEP WRANGLER,0.0,288900
2,Snohomish,47.82557,-122.29245,2022,KIA,NIRO,PHEV,26.0,Not eligible,2022 KIA NIRO,0.0,873800


#### Median Family Income data for each County

In [22]:
# Load MedFamInc.csv file
MedFamInc = pd.read_csv('Resources/MedFamInc.csv')

MedFamInc.head()

,County,Value (Dollars),Rank within US
0,NaN,NaN,(of 3139 counties)
1,Washington,"114,068",9 of 52
2,United States,"96,922",NaN
3,King County,"154,490",24
4,Snohomish County,"127,475",110


In [23]:
# Remove top 2 rows
MedFamInc = MedFamInc.iloc[3:].reset_index(drop=True)

# Clean and standardize county names in both DataFrames
MedFamInc['County'] = (
    MedFamInc['County']
    .str.strip()                     # remove leading/trailing spaces
    .str.lower()                     # convert to lowercase
    .str.replace(' county', '', regex=False)  # remove the word ' county'
    .str.strip()                     # remove any leftover spaces
)
pop_ev['County'] = pop_ev['County'].str.strip().str.lower()

# Convert 'Value (Dollars)' to integers
MedFamInc['Value (Dollars)'] = (
    MedFamInc['Value (Dollars)']
    .replace({',': ''}, regex=True)
    .replace('', pd.NA)
    .astype('Int64')
)

# Merge MedFamInc with pop_ev
pop_ev = pop_ev.merge(
    MedFamInc[['County', 'Value (Dollars)']],
    on='County',
    how='left'
)

pop_ev.head(3)

,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,Electric Range,CAFV Eligibility,Year_Make_Model,Base MSRP,POP_2025,Value (Dollars)
0,yakima,46.60464,-120.50729,2019,TESLA,MODEL 3,BEV,220.0,Eligible,2019 TESLA MODEL 3,0.0,264650,76662
1,kitsap,47.50524,-122.68471,2024,JEEP,WRANGLER,PHEV,21.0,Not eligible,2024 JEEP WRANGLER,0.0,288900,114590
2,snohomish,47.82557,-122.29245,2022,KIA,NIRO,PHEV,26.0,Not eligible,2022 KIA NIRO,0.0,873800,127475


#### Unique EV Models: 'Year_Make_Model'

In [24]:
ev_price = pop_ev[["Year_Make_Model", "Vehicle Type", "Base MSRP", "Electric Range"]]
# ev_price.head()

# Get unique combinations and sort
ev_price_unique = (
    ev_price
    .drop_duplicates(subset=["Year_Make_Model", "Vehicle Type"])
    .sort_values(by=["Year_Make_Model", "Vehicle Type"])
    .reset_index(drop=True)
)

# Convert to csv
ev_price_unique.to_csv('Resources/ev_price_unique.csv', index = False)

In [28]:
# Number of Unique models
ev_type = pop_ev['Year_Make_Model'].nunique()
print(f'\nUnique EV models (Year-Make-Model): {ev_type}')

# Range of years in data
years = pop_ev['Year_Make_Model'].str.extract(r'(\d{4})')[0].astype(int)
print(f'Years Range: {years.min()} - {years.max()}')



Unique EV models (Year-Make-Model): 639
Years Range: 2000 - 2026


In [29]:
# Retrieve ev_price_unique.csv file
ev_price_unique = pd.read_csv('Resources/ev_price_unique.csv')

print(f'\nUnique EV prices (MSRP): {ev_price_unique.shape[0]}\n')
ev_price_unique["Base MSRP"] = ev_price_unique["Base MSRP"].fillna(0).astype("int64")
ev_price_unique["Electric Range"] = ev_price_unique["Electric Range"].fillna(0).astype("int64")
ev_price_unique.tail()



Unique EV prices (MSRP): 658



,Year_Make_Model,Vehicle Type,Base MSRP,Electric Range
653,2026 KIA EV9,BEV,0,0
654,2026 TESLA MODEL S,BEV,0,0
655,2026 TESLA MODEL X,BEV,0,0
656,2026 TESLA MODEL Y,BEV,0,0
657,2026 VOLVO XC60,PHEV,0,35


In [30]:
# Missing MSRP, Electric Range data
ev_price_unique[ev_price_unique['Base MSRP']==0.0].count()

Year_Make_Model    630
Vehicle Type       630
Base MSRP          630
Electric Range     630
dtype: int64

#### BEV MSRP Data for the Years 2020-2025

In [32]:
# Filter rows where MSRP == 0 OR Electric Range == 0
ev_rng0_msrp0 = pop_ev[(pop_ev['Base MSRP'] == 0) | (pop_ev['Electric Range'] == 0)]
print(f'\nInstances with Missing MSRP or Electric Range data: {ev_rng0_msrp0.shape[0]}\n')



Instances with Missing MSRP or Electric Range data: 253868



In [34]:
import glob

# Get all CSV files in the folder
BEV_files = glob.glob("Resources/US_EV_Prices*.csv")

# Read and concatenate them
BEV_list = [pd.read_csv(BEV_f) for BEV_f in BEV_files]
BEV_price = pd.concat(BEV_list, ignore_index=True)

# Optional cleanup
BEV_price.columns = BEV_price.columns.str.strip()   # remove stray spaces
BEV_price = BEV_price.drop_duplicates()              # remove duplicate rows

# Save to a new CSV file
BEV_price.to_csv("Resources/BEV_price.csv", index=False)

print(f"Combined file saved as BEV_price.csv with {len(BEV_price)} rows")


Combined file saved as BEV_price.csv with 655 rows


In [35]:
# Retrieve BEV_price.csv file
BEV_price = pd.read_csv('Resources/BEV_price.csv')
print(BEV_price.info())
BEV_price.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Model              655 non-null    object 
 1   Base Price         654 non-null    object 
 2   Dest. Charge       639 non-null    object 
 3   Tax Credit         350 non-null    object 
 4   Effective Price    654 non-null    object 
 5   EPA range (miles)  639 non-null    float64
dtypes: float64(1), object(5)
memory usage: 30.8+ KB
None


,Model,Base Price,Dest. Charge,Tax Credit,Effective Price,EPA range (miles)
0,2021 MINI Cooper SE,"$29,900",$850,"$7,500","$23,250",110.0
1,2021 Nissan LEAF S (40 kWh),"$31,670",$950,"$7,500","$25,120",149.0


In [36]:
# Remove '$', ',', '*', 'NaN'
# Convert numerical data into integers

price_columns = ['Base Price', 'Dest. Charge', 'Tax Credit', 'Effective Price']

for price_cols in price_columns:
    BEV_price[price_cols] = BEV_price[price_cols].replace({'\$': '', ',': '', '\*': '', 'NaN': ''}, regex=True).replace('', pd.NA).astype('Int64')
BEV_price['EPA range (miles)'] = BEV_price['EPA range (miles)'].fillna(0).astype('int64')    
print(f'Instances of BEV price data: {BEV_price.shape[0]}\n')    
BEV_price.head(2)    

Instances of BEV price data: 655



,Model,Base Price,Dest. Charge,Tax Credit,Effective Price,EPA range (miles)
0,2021 MINI Cooper SE,29900,850,7500,23250,110
1,2021 Nissan LEAF S (40 kWh),31670,950,7500,25120,149


#### Extract Year-Make-Model without trim details

In [37]:
# Copy the original DataFrame
bev_price = BEV_price.copy()

# Extract first 4 words from 'Model' to group by
bev_price['Model_Group'] = bev_price['Model'].str.split().str[:4].str.join(' ')

# Group and compute the mean of numeric columns
bev = bev_price.groupby('Model_Group', as_index=False).mean(numeric_only=True)

# Convert all numeric columns to integers
bev = bev.round(0).astype(int, errors='ignore')

# Rename Model_Group back to Model
bev = bev.rename(columns={'Model_Group': 'Model'})

print(bev.shape)
bev.head()

(263, 6)


,Model,Base Price,Dest. Charge,Tax Credit,Effective Price,EPA range (miles)
0,2020 Jaguar I-PACE,69850,1025.0,7500.0,63375,234
1,2020 Kia Niro EV,39090,1120.0,7500.0,32710,239
2,2021 Audi e-tron,65900,1095.0,7500.0,59495,222
3,2021 Audi e-tron Sportback,69100,1070.0,7500.0,62670,218
4,2021 BMW i3,44450,995.0,7500.0,37945,153


#### Merge EV data with 'MSRP', 'Electric Range'

In [38]:
# Create normalized match keys
# Lowercase and collapse multiple spaces

bev['Year_clean'] = bev['Model'].str.extract(r'(\d{4})')
bev['Model_clean'] = (
    bev['Model']
    .str.replace(r'^\d{4}\s*', '', regex=True)  # remove year
    .str.lower()
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
# print(bev['Model_clean'].head(2))

pop_ev['Year_clean'] = pop_ev['Year'].astype(str)
pop_ev['Model_clean'] = (
    pop_ev['Year_Make_Model']
    .str.replace(r'^\d{4}\s*', '', regex=True)
    .str.lower()
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
# print(pop_ev['Model_clean'].head(2))


In [39]:
# Build a merge key that includes year
bev['merge_key'] = bev['Year_clean'] + ' ' + bev['Model_clean']
pop_ev['merge_key'] = pop_ev['Year_clean'] + ' ' + pop_ev['Model_clean']

# Merge directly
pop_ev_matched = pop_ev.merge(bev, on='merge_key', how='inner', suffixes=('', '_bev'))

# Inspect
print(f"\nMatches found: {len(pop_ev_matched)} of {len(pop_ev)} pop_ev entries\nUnmatched EVs: {len(pop_ev)-len(pop_ev_matched)}\n\n")
# pop_ev_matched[100: 102]


Matches found: 36982 of 257038 pop_ev entries
Unmatched EVs: 220056




In [40]:
# Find unmatched rows
pop_ev_unmatched = pop_ev[~pop_ev['merge_key'].isin(pop_ev_matched['merge_key'])]

# Stack (vstack / append) them to pop_ev_matched
pop_ev_all = pd.concat([pop_ev_matched, pop_ev_unmatched], ignore_index=True)

# Inspect
print(f"All entries after stacking: {len(pop_ev_all)} (should equal {len(pop_ev)})")
# pop_ev_all[40000: 40002]

All entries after stacking: 257038 (should equal 257038)


In [41]:
pop_ev_all = pop_ev_all.drop(['Year_clean', 'Model_clean', 'merge_key', 'Model_bev', 'Year_clean_bev', 'Model_clean_bev'], axis = 1)
pop_ev_all.columns

Index(['County', 'Latitude', 'Longitude', 'Year', 'Make', 'Model',
       'Vehicle Type', 'Electric Range', 'CAFV Eligibility', 'Year_Make_Model',
       'Base MSRP', 'POP_2025', 'Value (Dollars)', 'Base Price',
       'Dest. Charge', 'Tax Credit', 'Effective Price', 'EPA range (miles)'],
      dtype='object')

In [42]:

# Fill NaNs and empty strings
pop_ev_all = pop_ev_all.fillna(0).replace('', 0)

# Convert all numeric columns to int, EXCEPT Latitude & Longitude
for col in pop_ev_all.select_dtypes(include=['number']).columns:
    if col not in ['Latitude', 'Longitude']:
        pop_ev_all[col] = pop_ev_all[col].astype(int)

pop_ev_all[40000: 40002]

,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,Electric Range,CAFV Eligibility,Year_Make_Model,Base MSRP,POP_2025,Value (Dollars),Base Price,Dest. Charge,Tax Credit,Effective Price,EPA range (miles)
40000,spokane,47.63396,-117.36043,2023,NISSAN,ARIYA,BEV,0,Unknown,2023 NISSAN ARIYA,0,566000,94226,0,0,0,0,0
40001,island,48.23986,-122.40049,2025,VOLVO,XC90,PHEV,32,Eligible,2025 VOLVO XC90,0,88700,105012,0,0,0,0,0


In [43]:
pop_ev_all.loc[pop_ev_all['EPA range (miles)'] == 0, 'EPA range (miles)'] = pop_ev_all['Electric Range']
pop_ev_all.loc[pop_ev_all['Base Price'] == 0, 'Base Price'] = pop_ev_all['Base MSRP']
# pop_ev_all[40000: 40002]

In [44]:
pop_ev_all = pop_ev_all.drop(['Base MSRP', 'Electric Range'], axis = 1)
pop_ev_all.columns

Index(['County', 'Latitude', 'Longitude', 'Year', 'Make', 'Model',
       'Vehicle Type', 'CAFV Eligibility', 'Year_Make_Model', 'POP_2025',
       'Value (Dollars)', 'Base Price', 'Dest. Charge', 'Tax Credit',
       'Effective Price', 'EPA range (miles)'],
      dtype='object')

#### CAFV Eligibility

In [49]:
pop_ev_all.loc[pop_ev_all['EPA range (miles)'] >= 200, 'CAFV Eligibility'] = 'Eligible'

In [50]:
pop_ev_all = pop_ev_all.rename(columns={
    'EPA range (miles)': 'Electric Range',
    'Base Price': 'MSRP',
    'Value (Dollars)': 'Median Income'
})

In [51]:
pop_ev_all.to_csv('Resources/pop_ev_all.csv', index = False)

#### Subset of Master file with missing 'MSRP' , 'Electric Range'

In [52]:
pop_ev_all = pd.read_csv('Resources/pop_ev_all.csv')

# Filter rows where MSRP == 0 OR Electric Range == 0
ev_filtered = pop_ev_all[(pop_ev_all['MSRP'] == 0) | (pop_ev_all['Electric Range'] == 0)]

# Group by 'Make'
ev_make = ev_filtered.sort_values(by=['Make', 'Year'])
print(ev_make.shape)

ev_make.head(2)

(216889, 16)


,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,CAFV Eligibility,Year_Make_Model,POP_2025,Median Income,MSRP,Dest. Charge,Tax Credit,Effective Price,Electric Range
37434,king,47.67887,-122.2066,2024,ACURA,ZDX,BEV,Unknown,2024 ACURA ZDX,2411700,154490,0,0,0,0,0
37956,grant,47.13196,-119.2771,2024,ACURA,ZDX,BEV,Unknown,2024 ACURA ZDX,106250,79271,0,0,0,0,0


#### Instances with missing information - MSRP, Electric Range

In [53]:
ev_need_info = (
    ev_make
    .sort_values(by='Make')                # sort by Make first
    .drop_duplicates(subset='Year_Make_Model', keep='first')  # keep first row for each unique model
    .reset_index(drop=True)
)
print(ev_need_info.shape)
print(ev_need_info['Make'].nunique())
ev_need_info.head(2)

(587, 16)
45


,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,CAFV Eligibility,Year_Make_Model,POP_2025,Median Income,MSRP,Dest. Charge,Tax Credit,Effective Price,Electric Range
0,king,47.67887,-122.20660,2024,ACURA,ZDX,BEV,Unknown,2024 ACURA ZDX,2411700,154490,0,0,0,0,0
1,king,47.49408,-122.36178,2024,ALFA ROMEO,TONALE,PHEV,Eligible,2024 ALFA ROMEO TONALE,2411700,154490,0,0,0,0,33


In [56]:
ev_need_info.to_csv('Resources/ev_need_info.csv', index = False)

### Manually collected data for missing 'MSRP', 'Electric Range'

In [57]:
pop_ev_all = pd.read_csv('Resources/pop_ev_all.csv')
print(pop_ev_all.columns)
pop_ev_all.head(2)

Index(['County', 'Latitude', 'Longitude', 'Year', 'Make', 'Model',
       'Vehicle Type', 'CAFV Eligibility', 'Year_Make_Model', 'POP_2025',
       'Median Income', 'MSRP', 'Dest. Charge', 'Tax Credit',
       'Effective Price', 'Electric Range'],
      dtype='object')


,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,CAFV Eligibility,Year_Make_Model,POP_2025,Median Income,MSRP,Dest. Charge,Tax Credit,Effective Price,Electric Range
0,yakima,46.60464,-120.50729,2024,TESLA,MODEL 3,BEV,Eligible,2024 TESLA MODEL 3,264650,76662,44865,1515,5625,43568,306
1,king,47.53010,-122.03439,2021,AUDI,E-TRON,BEV,Eligible,2021 AUDI E-TRON,2411700,154490,65900,1095,7500,59495,222


In [58]:
ev_filled = pd.read_csv('Resources/ev_filled_info.csv')
print(ev_filled.columns)
ev_filled.head(2)

Index(['County', 'Latitude', 'Longitude', 'Year', 'Make', 'Model',
       'Vehicle Type', 'CAFV Eligibility', 'Year_Make_Model', 'POP_2025',
       'Median Income', 'MSRP', 'Dest. Charge', 'Tax Credit',
       'Effective Price', 'Electric Range', 'Source'],
      dtype='object')


,County,Latitude,Longitude,Year,Make,Model,Vehicle Type,CAFV Eligibility,Year_Make_Model,POP_2025,Median Income,MSRP,Dest. Charge,Tax Credit,Effective Price,Electric Range,Source
0,king,47.67887,-122.20660,2024,ACURA,ZDX,BEV,Unknown,2024 ACURA ZDX,2411700,154490,74500,0,0,0,278,api.marketcheck.com
1,king,47.49408,-122.36178,2024,ALFA ROMEO,TONALE,PHEV,Eligible,2024 ALFA ROMEO TONALE,2411700,154490,42995,0,0,0,33,Edmunds


In [59]:
# Merge files to bring in the missing values from ev_filled
ev_new = pop_ev_all.merge(
    ev_filled[['Year_Make_Model', 'MSRP', 'Electric Range']],
    on='Year_Make_Model',
    how='left',
    suffixes=('', '_filled')
)

# Fill missing values in ev with values from ev_filled
# Replace only zeros (not NaN) in ev['MSRP'] and ev['Electric Range']
ev_new.loc[ev_new['MSRP'] == 0, 'MSRP'] = ev_new.loc[ev_new['MSRP'] == 0, 'MSRP_filled']
ev_new.loc[ev_new['Electric Range'] == 0, 'Electric Range'] = ev_new.loc[ev_new['Electric Range'] == 0, 'Electric Range_filled']
ev_new.tail(10)
                                                                         
# Drop helper columns
ev_new = ev_new.drop(columns=['MSRP_filled', 'Electric Range_filled', 'Effective Price'])
ev_new.columns

Index(['County', 'Latitude', 'Longitude', 'Year', 'Make', 'Model',
       'Vehicle Type', 'CAFV Eligibility', 'Year_Make_Model', 'POP_2025',
       'Median Income', 'MSRP', 'Dest. Charge', 'Tax Credit',
       'Electric Range'],
      dtype='object')

#### Requiremnt for 'CAFV' Eligiblity over years 2000 to 2025

In [60]:
mask = (ev_new['CAFV Eligibility'] == 'Unknown') & (ev_new['Vehicle Type'].str.upper() == 'BEV')

condition1 = (ev_new['Electric Range'] >= 200) & (ev_new['Year'] >= 2019)
condition2 = (ev_new['Electric Range'] < 200) & (ev_new['Year'] < 2019)

# np.select on the filtered rows
ev_new.loc[mask, 'CAFV Eligibility'] = np.select(
    [condition1[mask], condition2[mask]],
    ['Eligible', 'Eligible'],
    default='Not Eligible'
)
print(f"List CAFV Eligibility = Unknown:\n{ev_new[ev_new['CAFV Eligibility'] == 'Unknown'].value_counts()}")

ev_new.loc[ev_new['CAFV Eligibility'].str.lower() != 'eligible', 'CAFV Eligibility'] = 'Not Eligible'

List CAFV Eligibility = Unknown:
Series([], Name: count, dtype: int64)


In [63]:
# ev_new.columns
ev_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257038 entries, 0 to 257037
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   County            257038 non-null  object 
 1   Latitude          257038 non-null  float64
 2   Longitude         257038 non-null  float64
 3   Year              257038 non-null  int64  
 4   Make              257038 non-null  object 
 5   Model             257038 non-null  object 
 6   Vehicle Type      257038 non-null  object 
 7   CAFV Eligibility  257038 non-null  object 
 8   Year_Make_Model   257038 non-null  object 
 9   POP_2025          257038 non-null  int64  
 10  Median Income     257038 non-null  int64  
 11  MSRP              257038 non-null  int64  
 12  Dest. Charge      257038 non-null  int64  
 13  Tax Credit        257038 non-null  int64  
 14  Electric Range    257038 non-null  int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 29.4+ MB


In [62]:
ev_new.to_csv('Resources/ev_new.csv', index = False)